## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Matara,LK,5.9485,80.5353,74.68,84,90,2.86,overcast clouds
1,1,Albany,US,42.6001,-73.9662,27.68,60,71,1.99,broken clouds
2,2,Barrow,US,71.2906,-156.7887,-25.58,68,0,11.50,clear sky
3,3,Avarua,CK,-21.2078,-159.7750,82.45,78,20,11.50,few clouds
4,4,Bengkulu,ID,-3.8004,102.2655,75.92,87,100,9.80,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.2078,-159.7750,82.45,78,20,11.50,few clouds
4,4,Bengkulu,ID,-3.8004,102.2655,75.92,87,100,9.80,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.20,70,2,17.45,clear sky
7,7,Kahului,US,20.8947,-156.4700,85.26,68,20,18.41,few clouds
10,10,Hilo,US,19.7297,-155.0900,75.25,92,100,5.75,heavy intensity rain
13,13,Souillac,MU,-20.5167,57.5167,78.37,85,72,10.49,broken clouds
22,22,Puerto Madero,MX,14.7208,-92.4215,86.20,62,40,8.05,scattered clouds
37,37,Faanui,PF,-16.4833,-151.7500,81.55,75,100,20.65,overcast clouds
41,41,Bandarbeyla,SO,9.4942,50.8122,75.31,74,73,13.76,broken clouds
42,42,Cidreira,BR,-30.1811,-50.2056,77.04,63,1,15.77,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_id                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# there are no empty rows, convert preferred_cities_df to clean_df
clean_df = preferred_cities_df
clean_df.head()

,City_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.2078,-159.7750,82.45,78,20,11.50,few clouds
4,4,Bengkulu,ID,-3.8004,102.2655,75.92,87,100,9.80,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.20,70,2,17.45,clear sky
7,7,Kahului,US,20.8947,-156.4700,85.26,68,20,18.41,few clouds
10,10,Hilo,US,19.7297,-155.0900,75.25,92,100,5.75,heavy intensity rain


In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
4,Bengkulu,ID,75.92,overcast clouds,-3.8004,102.2655,
6,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,
7,Kahului,US,85.26,few clouds,20.8947,-156.4700,
10,Hilo,US,75.25,heavy intensity rain,19.7297,-155.0900,


In [57]:
# 6a. Set parameters to search for hotels with 5000 meters.
import numpy as np

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found...skipping")

    

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [68]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   178
Country                178
Max Temp               178
Current Description    178
Lat                    178
Lng                    178
Hotel Name             178
dtype: int64

In [66]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [70]:
clean_hotel_df.

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,Paradise Inn
4,Bengkulu,ID,75.92,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
6,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,People ThankYou
7,Kahului,US,85.26,few clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Hilo,US,75.25,heavy intensity rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [73]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}"and"{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))